In [7]:
print("Hello world!")

## https://www.geeksforgeeks.org/select-rows-columns-by-name-or-index-in-pandas-dataframe-using-loc-iloc/
import pandas as pd

# Load the CSV file
df = pd.read_csv("U15630142_2024_2024_filtered.csv" , sep=";")#on_bad_lines='skip')


df = df[df['DataDiscriminator'] == 'Order'] # remove other types of trades like subtotal and total, focus on orders only
df = df.loc[:, ["Symbol", "Date/Time", "Quantity", "T. Price", "Proceeds"]]
# Display the first few rows (optional)
#print(df.head())

# Filtering columns (selecting specific ones)
#selected_columns = ["column_name1", "column_name2"]  # Replace with actual column names
#df_filtered_columns = df[selected_columns]

# Filtering rows (by text value in a specific column)
#filtered_rows = df[df["Trades"]]  # Replace with actual column name and text
#result_loc = df.loc[0, 'Name']  # Select value at row 0 and column 'name'
# Filtering both rows and columns together
#df_filtered = df_filtered_columns[df_filtered_columns["column_name"] == "desired_text_value"]

# Display the result
print("\n\n")
#print(df.loc[1:10,"Symbol"])
print(df)



# Save the filtered data if needed
#df_filtered.to_csv("filtered_output.csv", index=False)



Hello world!



    Symbol             Date/Time  Quantity  T. Price  Proceeds
0       AI  2024-09-05, 04:34:06        15   18.8400  -282.600
1       AI  2024-09-06, 09:41:52       -15   21.3900   320.850
3     ANRO  2024-10-24, 00:46:59        25    4.5260  -113.150
4     ANRO  2024-10-25, 10:06:56       -25    4.0200   100.500
6     COIN  2024-08-22, 09:50:40       -10  202.5700  2025.700
..     ...                   ...       ...       ...       ...
154   TSLA  2024-11-08, 10:19:43       -10  312.9400  3129.400
155   TSLA  2024-11-15, 02:31:12         5  306.4500 -1532.250
156   TSLA  2024-11-18, 02:03:42        -5  336.9000  1684.500
157   TSLA  2024-12-05, 09:32:00        10  364.7698 -3647.698
158   TSLA  2024-12-06, 16:01:25       -10  389.2800  3892.800

[150 rows x 5 columns]


In [13]:
print(df.loc[0].values) # that makes a list of the values


def normalizeTrades(df): # we need to create bucket + create trades at the same time since every new sell needs to much the last buy bucket so there is no need to compare dates
    tickers = {"AI":1} # holds {"TSLA":[df.loc[i].values, df.loc[i+1].values, ..] , .. } --> each ticker holds buckets, each bucket has older date as we go to the end of the list

    print(row.values)

    # bucket format is list with items: 0:Symbol, 1:datetime, 2:quantity, 3: buy price, 4:total cost
    outputColumns = ["Symbol", "startDate", "endDate", "buyPrice", "sellPrice", "Quantity" ]
    df_out = pd.DataFrame(columns=outputColumns)
    #new_row = {'id': row['id'], 'name_upper': row['name'].upper()}
    #df_out = df_out.append(new_row, ignore_index=True)

    for index, row in df.iterrows():

        # Bucket generation
        if row["Quantity" > 0]: # BUY trade
            if row["Symbol"] in tickers.keys(): # then its already created, just add the new bucket at the end of bucketlist
                tickers[row["Symbol"]].append(row.values)
            else: # create initial ticker and the bucketlist
                bucketlist = []
                bucketlist.append(row.values) # put it at the end of a list since is the newest
                tickers[row["Symbol"]] = bucketlist

            

            # Trade generation
            if row["Quantity"] < 0: # SELL trade 
                if row["Symbol"] in tickers.keys(): # to be sure + filter older buy that are not inlcuded in the selected time window
                    tickerBlist = tickers[row["Symbol"]] # list of buckets
                    sellQt = abs(row["Quantity"]) # absolute value for using later
                    while sellQt > 0: # while we haven't consumed all the sell trade using the buckets, keep going to the next one
                        if len(tickerBlist) > 0: # at least 1 bucket
                            bucket = tickerBlist.pop()
                            bucketQt = bucket[2]  # 2: quantity
                            if (bucketQt - sellQt == 0): # best case, bucket is poped ,sellqt covered 
                                # todo construct trade
                                newRow = {"Symbol" : row["Symbol"], "startDate" : bucket[1], "endDate" : row["Date/Time"], "buyPrice" : bucket[3], "sellPrice": row["T. Price"], "Quantity": sellQt}
                                df_out = df_out.append(newRow, ignore_index = True)
                                sellQt= 0
                                break
                            elif (bucketQt - sellQt > 0): # we have remaining stocks in the bucket, put it back for the next sell order
                                bucket[2] = bucketQt - sellQt # update leftover quantity
                                tickerBlist.append(bucket) # put the back back
                                tickers[row["Symbol"]] = tickerBlist # update dictionary
                                # todo construct trade
                                newRow = {"Symbol" : row["Symbol"], "startDate" : bucket[1], "endDate" : row["Date/Time"], "buyPrice" : bucket[3], "sellPrice": row["T. Price"], "Quantity": sellQt}
                                df_out = df_out.append(newRow, ignore_index = True)
                                sellQt = 0 # just in case
                                break # stop loop, its done
                            else: # bucketQt - sellQt < 0, meaning that we need the next bucket(s) to consume sellQt
                                sellQt = sellQt - bucketQt
                                # todo construct partial trade
                                newRow = {"Symbol" : row["Symbol"], "startDate" : bucket[1], "endDate" : row["Date/Time"], "buyPrice" : bucket[3], "sellPrice": row["T. Price"], "Quantity": bucketQt}
                                df_out = df_out.append(newRow, ignore_index = True)

                        else: 
                            print("error: not enough buckets to fill sell order: ", row, "  |  Qt left: ", sellQt)

                        #todo if Qt not enough, cancel creation of the specific trade
            
        

    return 

normalizeTrades(df)

['AI' '2024-09-05, 04:34:06' np.int64(15) np.float64(18.84)
 np.float64(-282.6)]
